# Poseidon TCP Conflict Study Advanced Topology

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for performing iperf3 tests on short rtt fat pipe networks.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [FABlib API](https://learn.fabric-testbed.net/docs/fablib/fablib.html)

## Basic FABRIC Slice Configuration

In [ ]:
import json
import traceback
from time import sleep
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

In [ ]:
try:
    #fablib.list_sites(fields=["name", "cores_available", "ram_available", "nic_basic_available", "nic_connectx_5_available", "nic_connectx_6_available"])
    fablib.list_sites(fields=["name", "cores_available", "ram_available", "nic_connectx_6_available"])
    #fablib.list_sites()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# we will use CX5 to generate traffic so need sites that have CX6 for this example.
cx6_column_name = 'nic_connectx_6_available'

# find two sites with available ConnectX-6 
sites = fablib.get_random_sites(count=2, filter_function=lambda x: x[cx6_column_name] > 0)
print(f"Sites: {sites}")

In [ ]:
# find two sites with available ConnectX-6 
sites = fablib.get_random_sites(count=2, filter_function=lambda x: x[cx6_column_name] > 1)
print(f"Sites: {sites}")

## Configure Slice Parameters

In [ ]:
# Slice 
slice_name = 'PoSeiDon-TCP-Experiment-ConnectX6-Numa-50Gbps'

# Routers
r1_name = "poseidon-router-1"
r2_name = "poseidon-router-2"

net_r1_r2_name = "net_r1_r2"
subnet_r1_r2 = "192.168.1.0/30"

router_cores = 20
router_ram = 32
router_disk = 16

# Hosts
h1_name = "poseidon-sender-1"
h2_name = "poseidon-sender-2"

net_h1_r1_name = 'net_h1_r1'
subnet_h1_r1 = "192.168.2.0/30"

net_h2_r1_name = 'net_h2_r1'
subnet_h2_r1 = "192.168.3.0/30"

h3_name = "poseidon-receiver-1"
h4_name = "poseidon-receiver-2"

net_h3_r2_name = 'net_h3_r2'
subnet_h3_r2 = "192.168.4.0/30"

net_h4_r2_name = 'net_h4_r2'
subnet_h4_r2 = "192.168.5.0/30"

host_cores = 20
host_ram = 32
host_disk = 16

# Sites TACC STAR NCSA MICH DALL MAX MASS WASH CLEM SALT UTAH UCSD FIU
r1_site = 'INDI'
r2_site = 'PSC'
h1_site = 'FIU' # MICH UTAH TACC WASH NCSA DALL MAX MASS SALT STAR
h2_site = 'FIU' # MICH UTAH TACC WASH NCSA DALL MAX MASS SALT STAR
h3_site = 'MAX' # MICH UTAH TACC WASH NCSA DALL MAX MASS SALT STAR
h4_site = 'MAX' # MICH UTAH TACC WASH NCSA DALL MAX MASS SALT STAR

# All node properties
#username = 'ubuntu'
os_image = 'default_ubuntu_20'
#os_image = 'docker_ubuntu_20'
nvme_name="nme1"

### Create the Slice

In [ ]:
try:
    #Create Slice
    fabric_slice = fablib.new_slice(name=slice_name)
    
    # Add poseidon r1 node
    r1 = fabric_slice.add_node(
                    name=r1_name,
                    site=r1_site,
                    image=os_image,
                    cores=router_cores,
                    ram=router_ram,
                    disk=router_disk)
    
    r1_h1_h2_nic = r1.add_component(model='NIC_ConnectX_6', name="r1_h1_h2_nic")
    r1_iface_to_h1 = r1_h1_h2_nic.get_interfaces()[0]
    
    r1_iface_to_h2 = r1_h1_h2_nic.get_interfaces()[1]
    
    r1_r2_nic = r1.add_component(model='NIC_ConnectX_6', name="r1_r2_nic")
    r1_iface_to_r2 = r1_r2_nic.get_interfaces()[0]
    
    # Add poseidon r2 node
    r2 = fabric_slice.add_node(
                    name=r2_name,
                    site=r2_site,
                    image=os_image,
                    cores=router_cores,
                    ram=router_ram,
                    disk=router_disk)
    
    r2_h3_h4_nic = r2.add_component(model='NIC_ConnectX_6', name="r2_h3_h4_nic")
    r2_iface_to_h3 = r2_h3_h4_nic.get_interfaces()[0]
    
    r2_iface_to_h4 = r2_h3_h4_nic.get_interfaces()[1]
    
    r2_r1_nic = r2.add_component(model='NIC_ConnectX_6', name="r2_r1_nic")
    r2_iface_to_r1 = r2_r1_nic.get_interfaces()[0]

    # Add poseidon host node h1
    h1 = fabric_slice.add_node(
                    name=h1_name,
                    site=h1_site,
                    image=os_image,
                    cores=host_cores,
                    ram=host_ram,
                    disk=host_disk)
    
    h1_iface = h1.add_component(model='NIC_ConnectX_6', name="h1_nic").get_interfaces()[0]
    
    # Add poseidon host node h2
    h2 = fabric_slice.add_node(
                    name=h2_name,
                    site=h2_site,
                    image=os_image,
                    cores=host_cores,
                    ram=host_ram,
                    disk=host_disk)
    
    h2_iface = h2.add_component(model='NIC_ConnectX_6', name="h2_nic").get_interfaces()[0]
    
    # Add poseidon host node h3
    h3 = fabric_slice.add_node(
                    name=h3_name,
                    site=h3_site,
                    image=os_image,
                    cores=host_cores,
                    ram=host_ram,
                    disk=host_disk)
    
    h3_iface = h3.add_component(model='NIC_ConnectX_6', name="h3_nic").get_interfaces()[0]

    
    # Add poseidon host node h4
    h4 = fabric_slice.add_node(
                    name=h4_name,
                    site=h4_site,
                    image=os_image,
                    cores=host_cores,
                    ram=host_ram,
                    disk=host_disk)
    
    h4_iface = h4.add_component(model='NIC_ConnectX_6', name="h4_nic").get_interfaces()[0]
    
    #Add host networks 
    host_net_h1_r1 = fabric_slice.add_l2network(name=net_h1_r1_name, interfaces=[r1_iface_to_h1, h1_iface])
    host_net_h2_r1 = fabric_slice.add_l2network(name=net_h2_r1_name, interfaces=[r1_iface_to_h2, h2_iface])
    host_net_h3_r2 = fabric_slice.add_l2network(name=net_h3_r2_name, interfaces=[r2_iface_to_h3, h3_iface])
    host_net_h4_r2 = fabric_slice.add_l2network(name=net_h4_r2_name, interfaces=[r2_iface_to_h4, h4_iface])
    host_net_r1_r2 = fabric_slice.add_l2network(name=net_r1_r2_name, interfaces=[r1_iface_to_r2, r2_iface_to_r1])

    for n in fabric_slice.get_nodes():
        n.add_component(model='NVME_P4510', name=nvme_name)
    
    #Submit slice Request
    fabric_slice.submit(wait=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

## Setup NVME Drives

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for fabric_node in fabric_slice.get_nodes():
        nvme = fabric_node.get_component(nvme_name)
        nvme.show();
        nvme.configure_nvme();
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()      

## Install Multithreaded iPerf3

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for fabric_node in fabric_slice.get_nodes():
        fabric_node.upload_file("iperf3.sh", "/home/ubuntu/iperf3.sh")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()        

In [ ]:
try:
    config_command = "sudo sh /home/ubuntu/iperf3.sh"
    
    #Create execute threads
    execute_threads = {}
    for node in fabric_slice.get_nodes():
        print(f"Starting install on node {node.get_name()}")
        execute_threads[node] = node.execute_thread(config_command)
        
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()
        print(stdout)
        print(stderr)

    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

## Upload Post Boot Script and install necessary apps in Host 1 and 2

In [ ]:
from ipaddress import ip_address, IPv6Address

try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for fabric_node in fabric_slice.get_nodes():
        fabric_node.upload_file("bbrv2_setup.sh", "/home/ubuntu/bbrv2_setup.sh")
        fabric_node.upload_file("kernel-+v2alpha+a23c4bb59e0c+FABRIC.tar.gz2", "/home/ubuntu/kernel-+v2alpha+a23c4bb59e0c+FABRIC.tar.gz2")
        #if type(ip_address(fabric_node.get_management_ip())) is IPv6Address:
        #    fabric_node.execute("sudo sed -i 's/nameserver/nameserver 2a01:4f9:c010:3f02::1\nnameserver 2a00:1098:2c::1\nnameserver 2a00:1098:2b::1\nnameserver/' /etc/resolv.conf", quiet=True)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Install Post Boot Script

In [ ]:
try:
    config_command = "sudo bash /home/ubuntu/bbrv2_setup.sh"
    
    #Create execute threads
    execute_threads = {}
    for node in fabric_slice.get_nodes():
        print(f"Starting config on node {node.get_name()}")
        execute_threads[node] = node.execute_thread(config_command)
        
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()
        print(stdout)
        print(stderr)

    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

## Upload BBRv3 Post Boot Script and install necessary apps in Host 1 and 2¶

In [ ]:
from ipaddress import ip_address, IPv6Address

try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for fabric_node in fabric_slice.get_nodes():
        fabric_node.upload_file("bbr3_setup.sh", "/home/ubuntu/bbr3_setup.sh")
        #if type(ip_address(fabric_node.get_management_ip())) is IPv6Address:
        #    fabric_node.execute("sudo sed -i 's/nameserver/nameserver 2a01:4f9:c010:3f02::1\nnameserver 2a00:1098:2c::1\nnameserver 2a00:1098:2b::1\nnameserver/' /etc/resolv.conf", quiet=True)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Install Post Boot Script for BBR3

In [ ]:
try:
    config_command = "sudo bash /home/ubuntu/bbr3_setup.sh"
    
    #Create execute threads
    execute_threads = {}
    for node in fabric_slice.get_nodes():
        print(f"Starting config on node {node.get_name()}")
        execute_threads[node] = node.execute_thread(config_command)
        
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()
        print(stdout)
        print(stderr)

    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

## Reboot Nodes To Load New Kernel and install apps

In [ ]:
try:
    config_command = "sudo reboot"
    
    #Create execute threads
    execute_threads = {}
    for node in fabric_slice.get_nodes():
        print(f"Rebooting node {node.get_name()}")
        execute_threads[node] = node.execute_thread(config_command)
        
    #Wait for results from threads
    for node,thread in execute_threads.items():
        print(f"Waiting for result from node {node.get_name()}")
        stdout,stderr = thread.result()
    
    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    config_command = "ls -al"
    
    #Check if nodes are up
    for node in fabric_slice.get_nodes():
        print(f"Waiting for node {node.get_name()}")
        stdout,stderr = node.execute(config_command)
        print(stdout)
        print(stderr)
        
    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

## Perform CPU Pinning and Numa Tuning for all nodes 

In [ ]:
node_nics = {
    r1_name: ["r1_h1_h2_nic", "r1_r2_nic"],
    r2_name: ["r2_r1_nic", "r2_h3_h4_nic"],
    h1_name: ["h1_nic"],
    h2_name: ["h2_nic"],
    h3_name: ["h3_nic"],
    h4_name: ["h4_nic"]
}

try:
    for node_name in node_nics:
        node = fabric_slice.get_node(node_name)
        node_cores = node.get_cores()
        node_nic_names = node_nics[node_name]
        node_cores_per_nic = node_cores // len(node_nic_names)
        
        for i in range(len(node_nic_names)):
            # Select the cpus to pin based on the number of nics active
            cpu_range = f"{i*node_cores_per_nic}-{(i+1)*node_cores_per_nic - 1}"
            
            # Pass in the range of the vCPUs to be pinned 
            node.pin_cpu(component_name=node_nic_names[i], cpu_range_to_pin=cpu_range)
            #node.pin_cpu(component_name=node_nic_names[i])
        
        # Pin memmory for VM to same Numa node as the components
        try:
            node.numa_tune()
        except Exception as e:
            print(f"Exception: {e}")
    
        # Reboot the VM
        node.os_reboot()
except Exception as e:
    print(f"Exception: {e}")


In [ ]:
try:
    config_command = "ls -al"
    
    #Check if nodes are up
    for node in fabric_slice.get_nodes():
        print(f"Waiting for node {node.get_name()}")
        stdout,stderr = node.execute(config_command)
        print(stdout)
        print(stderr)
        
    print("All Done!")
except Exception as e:
    print(f"Exception: {e}")

## Configure Poseidon Router 1

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    r1 = fabric_slice.get_node(name=r1_name)
    
    stdout1, stderr = r1.execute('sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
        
    r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
    r1_r2_os_iface = r1_iface_r2.get_os_interface()
    stdout, stderr = r1.execute(f'sudo ip link set dev {r1_r2_os_iface} up', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip link set mtu 8900 dev {r1_r2_os_iface}', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip addr add 192.168.1.1/30 dev {r1_r2_os_iface}', quiet=True)
    #stdout, stderr = r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root fq', quiet=True)
    
    r1_iface_h1 = r1.get_interface(network_name=net_h1_r1_name)
    r1_h1_os_iface = r1_iface_h1.get_os_interface()
    stdout, stderr = r1.execute(f'sudo ip link set dev {r1_h1_os_iface} up', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip link set mtu 8900 dev {r1_h1_os_iface}', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip addr add 192.168.2.1/30 dev {r1_h1_os_iface}', quiet=True)
    #stdout, stderr = r1.execute(f'sudo tc qdisc add dev {r1_h1_os_iface} root fq', quiet=True)
    
    r1_iface_h2 = r1.get_interface(network_name=net_h2_r1_name)
    r1_h2_os_iface = r1_iface_h2.get_os_interface()
    stdout, stderr = r1.execute(f'sudo ip link set dev {r1_h2_os_iface} up', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip link set mtu 8900 dev {r1_h2_os_iface}', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip addr add 192.168.3.1/30 dev {r1_h2_os_iface}', quiet=True)
    #stdout, stderr = r1.execute(f'sudo tc qdisc add dev {r1_h2_os_iface} root fq', quiet=True)
    
    stdout, stderr = r1.execute(f'sudo ip route add 192.168.4.0/30 via 192.168.1.2', quiet=True)
    stdout, stderr = r1.execute(f'sudo ip route add 192.168.5.0/30 via 192.168.1.2', quiet=True)
    
    stdout, stderr = r1.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = r1.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Configure Poseidon Router 2

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    r2 = fabric_slice.get_node(name=r2_name)
    
    stdout1, stderr = r2.execute('sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)
        
    r2_iface_r1 = r2.get_interface(network_name=net_r1_r2_name)    
    r2_r1_os_iface = r2_iface_r1.get_os_interface()
    stdout, stderr = r2.execute(f'sudo ip link set dev {r2_r1_os_iface} up', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip link set mtu 8900 dev {r2_r1_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip addr add 192.168.1.2/30 dev {r2_r1_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo tc qdisc add dev {r2_r1_os_iface} root fq', quiet=True)
    
    r2_iface_h3 = r2.get_interface(network_name=net_h3_r2_name)
    r2_h3_os_iface = r2_iface_h3.get_os_interface()
    stdout, stderr = r2.execute(f'sudo ip link set dev {r2_h3_os_iface} up', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip link set mtu 8900 dev {r2_h3_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip addr add 192.168.4.1/30 dev {r2_h3_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo tc qdisc add dev {r2_h3_os_iface} root fq', quiet=True)
    
    r2_iface_h4 = r2.get_interface(network_name=net_h4_r2_name)
    r2_h4_os_iface = r2_iface_h4.get_os_interface()
    stdout, stderr = r2.execute(f'sudo ip link set dev {r2_h4_os_iface} up', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip link set mtu 8900 dev {r2_h4_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip addr add 192.168.5.1/30 dev {r2_h4_os_iface}', quiet=True)
    stdout, stderr = r2.execute(f'sudo tc qdisc add dev {r2_h4_os_iface} root fq', quiet=True)
    
    stdout, stderr = r2.execute(f'sudo ip route add 192.168.2.0/30 via 192.168.1.1', quiet=True)
    stdout, stderr = r2.execute(f'sudo ip route add 192.168.3.0/30 via 192.168.1.1', quiet=True)
    
    stdout, stderr = r2.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = r2.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Configure Poseidon Host 1

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host1 = fabric_slice.get_node(name=h1_name)
    
    host1_iface = host1.get_interface(network_name=net_h1_r1_name)
    
    host1_os_iface = host1_iface.get_os_interface()

    stdout, stderr = host1.execute(f'sudo ip link set dev {host1_os_iface} up', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip link set mtu 8900 dev {host1_os_iface}', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip addr add 192.168.2.2/30 dev {host1_os_iface}', quiet=True)
    stdout, stderr = host1.execute(f'sudo tc qdisc add dev {host1_os_iface} root fq', quiet=True)
    
    stdout, stderr = host1.execute(f'sudo ip route add 192.168.1.0/30 via 192.168.2.1', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip route add 192.168.3.0/30 via 192.168.2.1', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip route add 192.168.4.0/30 via 192.168.2.1', quiet=True)
    stdout, stderr = host1.execute(f'sudo ip route add 192.168.5.0/30 via 192.168.2.1', quiet=True)
    
    stdout, stderr = host1.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = host1.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Configure Poseidon Host 2

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host2 = fabric_slice.get_node(name=h2_name)
    
    host2_iface = host2.get_interface(network_name=net_h2_r1_name)  
    
    host2_os_iface = host2_iface.get_os_interface()
    
    stdout, stderr = host2.execute(f'sudo ip link set dev {host2_os_iface} up', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip link set mtu 8900 dev {host2_os_iface}', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip addr add 192.168.3.2/30 dev {host2_os_iface}', quiet=True)
    stdout, stderr = host2.execute(f'sudo tc qdisc add dev {host2_os_iface} root fq', quiet=True)
    
    stdout, stderr = host2.execute(f'sudo ip route add 192.168.1.0/30 via 192.168.3.1', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip route add 192.168.2.0/30 via 192.168.3.1', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip route add 192.168.4.0/30 via 192.168.3.1', quiet=True)
    stdout, stderr = host2.execute(f'sudo ip route add 192.168.5.0/30 via 192.168.3.1', quiet=True)
    
    stdout, stderr = host2.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = host2.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Configure Poseidon Host 3

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host3 = fabric_slice.get_node(name=h3_name)
    
    host3_iface = host3.get_interface(network_name=net_h3_r2_name)
    
    host3_os_iface = host3_iface.get_os_interface()
    
    stdout, stderr = host3.execute(f'sudo ip link set dev {host3_os_iface} up', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip link set mtu 8900 dev {host3_os_iface}', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip addr add 192.168.4.2/30 dev {host3_os_iface}', quiet=True)
    stdout, stderr = host3.execute(f'sudo tc qdisc add dev {host3_os_iface} root fq', quiet=True)
    
    stdout, stderr = host3.execute(f'sudo ip route add 192.168.1.0/30 via 192.168.4.1', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip route add 192.168.2.0/30 via 192.168.4.1', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip route add 192.168.3.0/30 via 192.168.4.1', quiet=True)
    stdout, stderr = host3.execute(f'sudo ip route add 192.168.5.0/30 via 192.168.4.1', quiet=True)
    
    stdout, stderr = host3.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = host3.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Configure Poseidon Host 4

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host4 = fabric_slice.get_node(name=h4_name)
    
    host4_iface = host4.get_interface(network_name=net_h4_r2_name)  
     
    host4_os_iface = host4_iface.get_os_interface()
    
    stdout, stderr = host4.execute(f'sudo ip link set dev {host4_os_iface} up', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip link set mtu 8900 dev {host4_os_iface}', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip addr add 192.168.5.2/30 dev {host4_os_iface}', quiet=True)
    stdout, stderr = host4.execute(f'sudo tc qdisc add dev {host4_os_iface} root fq', quiet=True)
    
    stdout, stderr = host4.execute(f'sudo ip route add 192.168.1.0/30 via 192.168.5.1', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip route add 192.168.2.0/30 via 192.168.5.1', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip route add 192.168.3.0/30 via 192.168.5.1', quiet=True)
    stdout, stderr = host4.execute(f'sudo ip route add 192.168.4.0/30 via 192.168.5.1', quiet=True)
    
    stdout, stderr = host4.execute(f'ip route show', quiet=True)
    print (stdout)
    
    stdout, stderr = host4.execute(f'ip addr show', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Ping Router 1 from Router 2

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    router1 = fabric_slice.get_node(name=r1_name)
    stdout, stderr = router1.execute(f'ping -c 4 192.168.1.2', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Ping Host 3 from Host 1

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host1 = fabric_slice.get_node(name=h1_name)
    stdout, stderr = host1.execute(f'ping -c 4 192.168.4.2', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Ping Host 4 from Host 2

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    host2 = fabric_slice.get_node(name=h2_name)
    stdout, stderr = host2.execute(f'ping -c 4 192.168.5.2', quiet=True)
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## IPerf3

In [ ]:
stdout1, stderr1 = host3.execute("iperf3 -s -1 &"
                                 , quiet=True, output_file=f"{host3.get_name()}.log");

stdout2, stderr2 = host1.execute(f"iperf3 -c 192.168.4.2 -P 10 -t 30 -i 10 -O 10"
                                 , quiet=False, output_file=f"{host1.get_name()}.log");

## Upload Iperf3 Scripts to Host 1,2,3,4 and Router 1

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for node_name in [h1_name, h2_name]:
        node = fabric_slice.get_node(name=node_name)
        node.upload_file("start_iperf_client.sh", "/home/ubuntu/start_iperf_client.sh")
        node.upload_file("process_pcap.py", "/home/ubuntu/process_pcap.py")
        node.upload_file("active_rtt_probing.py", "/home/ubuntu/active_rtt_probing.py")

    for node_name in [h3_name, h4_name]:
        node = fabric_slice.get_node(name=node_name)
        node.upload_file("start_iperf_server.sh", "/home/ubuntu/start_iperf_server.sh")

    #uploading to Router #1
    node = fabric_slice.get_node(r1_name)
    node.upload_file("buffer_status_fifo.py", "/home/ubuntu/buffer_status_fifo.py")
    node.upload_file("buffer_status_fq_codel.py", "/home/ubuntu/buffer_status_fq_codel.py")
    node.upload_file("buffer_status_red.py", "/home/ubuntu/buffer_status_red.py")
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Extend Slice's Life

In [ ]:
from pytz import timezone
import datetime
end_date = (datetime.datetime.now(timezone("America/Los_Angeles")) + datetime.timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    fabric_slice.renew(end_date)
    fabric_slice = fablib.get_slice(name=slice_name)
    print(f'New lease end time: {fabric_slice.get_lease_end()}')
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Install req apps

In [ ]:
fabric_slice = fablib.get_slice(name=slice_name)
host1 = fabric_slice.get_node(name=h1_name)    
stdout, stderr = host1.execute(f'pip install scapy', quiet=True)
stdout, stderr = host1.execute(f'pip install pandas', quiet=True)
stdout, stderr = host1.execute(f'sudo DEBIAN_FRONTEND=noninteractive apt-get -y install tshark', quiet=True)
stdout, stderr = host1.execute(f'sudo apt-get install -y dos2unix', quiet=True)
stdout, stderr = host1.execute(f'dos2unix start_iperf_client.sh', quiet=True)


host2 = fabric_slice.get_node(name=h2_name)    
stdout, stderr = host2.execute(f'pip install scapy', quiet=True)
stdout, stderr = host2.execute(f'pip install pandas', quiet=True)
stdout, stderr = host2.execute(f'sudo DEBIAN_FRONTEND=noninteractive apt-get -y install tshark', quiet=True)
stdout, stderr = host2.execute(f'sudo apt-get install -y dos2unix', quiet=True)
stdout, stderr = host2.execute(f'dos2unix start_iperf_client.sh', quiet=True)

router1 = fabric_slice.get_node(name=r1_name)    
stdout, stderr = router1.execute(f'pip install pandas', quiet=True)

host3 = fabric_slice.get_node(name=h3_name)
stdout, stderr = host3.execute(f'sudo apt-get install -y dos2unix', quiet=True)
stdout, stderr = host3.execute(f'dos2unix start_iperf_server.sh', quiet=True)

host4 = fabric_slice.get_node(name=h4_name)
stdout, stderr = host4.execute(f'sudo apt-get install -y dos2unix', quiet=True)
stdout, stderr = host4.execute(f'dos2unix start_iperf_server.sh', quiet=True)

## Speed and CCA configuration

In [ ]:
mtu_setting = 8900
rtt_setting = 0.80 #.083change this based on the ping reponse from earlier cells

speed_config = [
    {"tag": "1gbps", "speed": 1000000000, "processes": 10, "parallel_streams": 1},
    {"tag": "10gbps", "speed": 10000000000, "processes": 20, "parallel_streams": 5},
    {"tag": "25gbps", "speed": 25000000000, "processes": 25, "parallel_streams": 10},
    {"tag": "40gbps", "speed": 40000000000, "processes": 25, "parallel_streams": 10}
]

cca_config = [
    {"cca1": "bbr", "cca2": "cubic"},      # bbr3 is called as bbr in the bbr3 kernel
    {"cca1": "bbr1", "cca2": "cubic"},
    {"cca1": "bbr2", "cca2": "cubic"},
    {"cca1": "htcp", "cca2": "cubic"},
    {"cca1": "cubic", "cca2": "cubic"},
    {"cca1": "bbr", "cca2": "bbr"},        # bbr3 is called as bbr in the bbr3 kernel
    {"cca1": "bbr1", "cca2": "bbr1"},
    {"cca1": "bbr2", "cca2": "bbr2"}
    {"cca1": "htcp", "cca2": "htcp"}
]

## FIFO TESTS

In [ ]:
%%capture output
import traceback
from time import sleep, time

def run_with_timeout(node, cmd, timeout):
    try:
        print(f"Executing command on {node.get_name()}: {cmd}")
        thread = node.execute_thread(cmd)
        start_time = time()
        while thread.is_alive() and (time() - start_time) < timeout:
            sleep(1)
        if thread.is_alive():
            print(f"Command on {node.get_name()} timed out: {cmd}")
            thread.terminate()
            return None, None
        stdout, stderr = thread.result()
        return stdout, stderr
    except Exception as e:
        print(f"Exception while executing command on {node.get_name()}: {e}")
        return None, None


try:
    fabric_slice = fablib.get_slice(name=slice_name)
    
    runtime = 210
    
    for cca_conf in cca_config:
        cca1 = cca_conf["cca1"]
        cca2 = cca_conf["cca2"]
        
        for speed_conf in speed_config:
            speed = speed_conf["speed"]
            processes = speed_conf["processes"]
            streams = speed_conf["parallel_streams"]
            tag = speed_conf["tag"]

            bdp = speed * rtt_setting / 8.0 #this is in bytes
            
            #for mult in [0.5,1,2,4,8,16]:
            for mult in [1,2,4]:
                print(f"{cca1}-{cca2}-{tag}-fifo-{mult}bdp")
                
                applied_bdp = int(bdp * mult)
                packets = int(applied_bdp / mtu_setting) #mtu is 8900 bytes
                
                print(f"Configuring buffer on node {r1_name}")
                r1 = fabric_slice.get_node(name=r1_name)              
                r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
                r1_r2_os_iface = r1_iface_r2.get_os_interface()
                    
                print(f"iface: {r1_r2_os_iface}, speed: {speed}, packets: {packets}")
                r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1: tbf rate {speed} burst 1570000 limit {applied_bdp}', quiet=False)
                
                #run 5 reps
                for rep in range(1,2):
                    print(f"Starting run {rep}")
                    #start servers
                    print(f"Starting servers on node {h3_name}")
                    node = fabric_slice.get_node(name=h3_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca1} {cca2} fifo {mult}bdp {rep} {processes}", quiet=True)

                    print(f"Starting servers on node {h4_name}")
                    node = fabric_slice.get_node(name=h4_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca2} {cca1} fifo {mult}bdp {rep} {processes}", quiet=True)

                    print(f"Starting queue script on router {r1_name}")
                    print(f'python3 /home/ubuntu/buffer_status.py {tag} {cca2} {cca1} fifo {mult}bdp {rep} {r1_r2_os_iface} {runtime+10} buffer_status.csv &')
                    stdout, stderr = r1.execute(f'python3 /home/ubuntu/buffer_status_fifo.py {tag} {cca1} {cca2} fifo {mult}bdp {rep} {r1_r2_os_iface} {runtime+10} buffer_status.csv > test.out 2>&1 &', quiet=True)
                    print(f"Executed queue script on router {r1_name}")
                    #print(stdout)
                    #print(stderr)

                    print(f"Starting active rtt probing in {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    stdout, stderr = node.execute(f'python3 /home/ubuntu/active_rtt_probing.py 192.168.2.2 192.168.4.2 {runtime} 100 {tag} {cca1} {cca2} fifo {mult}bdp {rep} > probe.out 2>&1 &', quiet=True)

                    print(f"Starting active rtt probing in {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    stdout, stderr = node.execute(f'python3 /home/ubuntu/active_rtt_probing.py 192.168.3.2 192.168.5.2 {runtime} 100 {tag} {cca2} {cca1} fifo {mult}bdp {rep} > probe.out 2>&1 &', quiet=True)

                    cmd_host_1 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} fifo {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    
                    cmd_host_2 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} fifo {mult}bdp {rep} {processes} 192.168.5.2 {streams}"

                    print(cmd_host_1)
                    print(cmd_host_2)
                    
                    #start clients via execute threads
                    execute_threads = {}
                    #execute_threads[node] = node.execute_thread(cmd_router_1)
                    print(f"Starting clients on node {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    execute_threads[node] = node.execute_thread(cmd_host_1)
                    print(f"Starting clients on node {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    execute_threads[node] = node.execute_thread(cmd_host_2)
                    
                    #Wait for results from threads
                    for node,thread in execute_threads.items():
                        print(f"Waiting for result from node {node.get_name()}")
                        stdout,stderr = thread.result()
                        #print(stdout)
                        #print(stderr)
                        
                    #iperf runs for 200seconds so we wait for 210
                    print("Waiting for iperf3 to finish for 210seconds")
                    sleep(runtime)

                    print(f"Cleaning and killing iperf3 and tcp_dump")                
                    #kill all iperf processes
                    kill_iperf_command = "pkill -f iperf3"

                    #kill all active probing processes
                    kill_active_probing = "pkill -f active_rtt_probing.py"
                    
                    #kill all tcp_dump processes
                    kill_tcpdump_command = "sudo pkill -f tcpdump"
                    
                    #kill all tcp_dump processes
                    kill_buffer_status_command = "sudo pkill -f buffer_status_fifo.py"

                    for node_name in [h1_name, h2_name, h3_name, h4_name, r1_name]:
                        node = fabric_slice.get_node(name=node_name)
                        print(f"Killing iperf3 and tcp_dump on node {node.get_name()}")
                        stdout,stderr = node.execute(kill_iperf_command)                        
                        stdout, stderr = node.execute(kill_tcpdump_command)  # Stop tcpdump gracefully by sending SIGINT sign
                        stdout, stderr = node.execute(kill_buffer_status_command)
                        stdout, stderr = node.execute(kill_active_probing)

                 
                    cmd_process_pcap_h1 = f"python3 /home/ubuntu/process_pcap.py /home/ubuntu/output/{tag}_{cca1}_{cca2}_fifo_{mult}bdp_{rep}"
                    cmd_process_pcap_h2 = f"python3 /home/ubuntu/process_pcap.py /home/ubuntu/output/{tag}_{cca2}_{cca1}_fifo_{mult}bdp_{rep}"
                   
                    node = fabric_slice.get_node(name=h1_name)
                    print(f"Starting pcap_process on node {node.get_name()}")
                    stdout, stderr = run_with_timeout(node, cmd_process_pcap_h1, 700)
                    if stdout is None and stderr is None:
                        print(f"Timeout or error occurred on node {node.get_name()} for command: {cmd_process_pcap_h1}")
                    else:
                        print(f"Completed pcap_process on node {node.get_name()}")

                    node = fabric_slice.get_node(name=h2_name)
                    print(f"Starting pcap_process on node {node.get_name()}")
                    stdout, stderr = run_with_timeout(node, cmd_process_pcap_h2, 700)
                    if stdout is None and stderr is None:
                        print(f"Timeout or error occurred on node {node.get_name()} for command: {cmd_process_pcap_h2}")
                    else:
                        print(f"Completed pcap_process on node {node.get_name()}")

    
    print("All done with fifo!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [ ]:
output.show()

## FQ_CODEL TESTS

In [ ]:
%%capture output
import traceback
from time import sleep, time

def run_with_timeout(node, cmd, timeout):
    try:
        print(f"Executing command on {node.get_name()}: {cmd}")
        thread = node.execute_thread(cmd)
        start_time = time()
        while thread.is_alive() and (time() - start_time) < timeout:
            sleep(1)
        if thread.is_alive():
            print(f"Command on {node.get_name()} timed out: {cmd}")
            thread.terminate()
            return None, None
        stdout, stderr = thread.result()
        return stdout, stderr
    except Exception as e:
        print(f"Exception while executing command on {node.get_name()}: {e}")
        return None, None


try:
    fabric_slice = fablib.get_slice(name=slice_name)
    runtime = 210
    
    for cca_conf in cca_config:
        cca1 = cca_conf["cca1"]
        cca2 = cca_conf["cca2"]
        
        for speed_conf in speed_config:
            speed = speed_conf["speed"]
            processes = speed_conf["processes"]
            streams = speed_conf["parallel_streams"]
            tag = speed_conf["tag"]

            bdp = speed * rtt_setting / 8.0 #this is in bytes
            
            #for mult in [0.5,1,2,4,8,16]:
            for mult in [1,2,4]:
                print(f"{cca1}-{cca2}-{tag}-fq_codel-{mult}bdp")
                
                applied_bdp = int(bdp * mult)
                packets = int(applied_bdp / mtu_setting) #mtu is 8900 bytes
                
                print(f"Configuring buffer on node {r1_name}")
                r1 = fabric_slice.get_node(name=r1_name)
        
                r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
                r1_r2_os_iface = r1_iface_r2.get_os_interface()

                print(f"iface: {r1_r2_os_iface}, speed: {speed}, packets: {packets}")
                r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1: tbf rate {speed} burst 1570000 limit {applied_bdp}', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 handle 10: fq_codel ecn limit {packets}', quiet=False)
                
                #run 5 reps
                for rep in range(1,2):
                    print(f"Starting run {rep}")
                    #start servers
                    print(f"Starting servers on node {h3_name}")
                    node = fabric_slice.get_node(name=h3_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca1} {cca2} fq_codel {mult}bdp {rep} {processes}", quiet=True)

                    print(f"Starting servers on node {h4_name}")
                    node = fabric_slice.get_node(name=h4_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca2} {cca1} fq_codel {mult}bdp {rep} {processes}", quiet=True)
                   
                    print(f"Starting queue script on router {r1_name}")
                    print(f'python3 /home/ubuntu/buffer_status.py {tag} {cca2} {cca1} fq_codel {mult}bdp {rep} {r1_r2_os_iface} {runtime+10} buffer_status.csv &')
                    stdout, stderr = r1.execute(f'python3 /home/ubuntu/buffer_status_fq_codel.py {tag} {cca1} {cca2} fq_codel {mult}bdp {rep} {r1_r2_os_iface} {runtime+10} > test.out 2>&1 &', quiet=True)
                    print(f"Executed queue script on router {r1_name}")
                    #print(stdout)
                    #print(stderr)

                    print(f"Starting active rtt probing in {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    stdout, stderr = node.execute(f'python3 /home/ubuntu/active_rtt_probing.py 192.168.2.2 192.168.4.2 {runtime} 100 {tag} {cca1} {cca2} fq_codel {mult}bdp {rep} > probe.out 2>&1 &', quiet=True)

                    print(f"Starting active rtt probing in {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    stdout, stderr = node.execute(f'python3 /home/ubuntu/active_rtt_probing.py 192.168.3.2 192.168.5.2 {runtime} 100 {tag} {cca2} {cca1} fq_codel {mult}bdp {rep} > probe.out 2>&1 &', quiet=True)
                    
                    cmd_host_1 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} fq_codel {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    
                    cmd_host_2 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} fq_codel {mult}bdp {rep} {processes} 192.168.5.2 {streams}"

                    #start clients via execute threads
                    execute_threads = {}
                    print(f"Starting clients on node {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    execute_threads[node] = node.execute_thread(cmd_host_1)
                    print(f"Starting clients on node {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    execute_threads[node] = node.execute_thread(cmd_host_2)

                    #Wait for results from threads
                    for node,thread in execute_threads.items():
                        #print(f"Waiting for result from node {node.get_name()}")
                        stdout,stderr = thread.result()

                    #iperf runs for 200seconds so we wait for 210
                    print("Waiting for iperf3 to finish for 210seconds")
                    sleep(runtime)

                    print(f"Cleaning and killing iperf3 and tcp_dump")                
                    #kill all iperf processes
                    kill_iperf_command = "pkill -f iperf3"

                    #kill all active probing processes
                    kill_active_probing = "pkill -f active_rtt_probing.py"
                    
                    #kill all tcp_dump processes
                    kill_tcpdump_command = "sudo pkill -f tcpdump"
                    
                    #kill all tcp_dump processes
                    kill_buffer_status_command = "sudo pkill -f buffer_status_fq_codel.py"

                    for node_name in [h1_name, h2_name, h3_name, h4_name, r1_name]:
                        node = fabric_slice.get_node(name=node_name)
                        print(f"Killing iperf3 and tcp_dump on node {node.get_name()}")
                        stdout,stderr = node.execute(kill_iperf_command)                        
                        stdout, stderr = node.execute(kill_tcpdump_command)  # Stop tcpdump gracefully by sending SIGINT sign
                        stdout, stderr = node.execute(kill_buffer_status_command)

                   
                    cmd_process_pcap_h1 = f"python3 /home/ubuntu/process_pcap.py /home/ubuntu/output/{tag}_{cca1}_{cca2}_fq_codel_{mult}bdp_{rep}"
                    cmd_process_pcap_h2 = f"python3 /home/ubuntu/process_pcap.py /home/ubuntu/output/{tag}_{cca2}_{cca1}_fq_codel_{mult}bdp_{rep}"
                   
                    node = fabric_slice.get_node(name=h1_name)
                    print(f"Starting pcap_process on node {node.get_name()}")
                    stdout, stderr = run_with_timeout(node, cmd_process_pcap_h1, 700)
                    if stdout is None and stderr is None:
                        print(f"Timeout or error occurred on node {node.get_name()} for command: {cmd_process_pcap_h1}")
                    else:
                        print(f"Completed pcap_process on node {node.get_name()}")

                    node = fabric_slice.get_node(name=h2_name)
                    print(f"Starting pcap_process on node {node.get_name()}")
                    stdout, stderr = run_with_timeout(node, cmd_process_pcap_h2, 700)
                    if stdout is None and stderr is None:
                        print(f"Timeout or error occurred on node {node.get_name()} for command: {cmd_process_pcap_h2}")
                    else:
                        print(f"Completed pcap_process on node {node.get_name()}")
                 
                    
    print("All done with fq_codel!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [ ]:
output.show()

## RED TESTS

In [ ]:
%%capture output
import traceback
from time import sleep, time

def run_with_timeout(node, cmd, timeout):
    try:
        print(f"Executing command on {node.get_name()}: {cmd}")
        thread = node.execute_thread(cmd)
        start_time = time()
        while thread.is_alive() and (time() - start_time) < timeout:
            sleep(1)
        if thread.is_alive():
            print(f"Command on {node.get_name()} timed out: {cmd}")
            thread.terminate()
            return None, None
        stdout, stderr = thread.result()
        return stdout, stderr
    except Exception as e:
        print(f"Exception while executing command on {node.get_name()}: {e}")
        return None, None


try:
    fabric_slice = fablib.get_slice(name=slice_name)
    runtime = 210
    
    for cca_conf in cca_config:
        cca1 = cca_conf["cca1"]
        cca2 = cca_conf["cca2"]
        
        for speed_conf in speed_config:
            speed = speed_conf["speed"]
            processes = speed_conf["processes"]
            streams = speed_conf["parallel_streams"]
            tag = speed_conf["tag"]

            bdp = speed * rtt_setting / 8.0 #this is in bytes
            
            #for mult in [0.5,1,2,4,8,16]:
            for mult in [1,2,4]:
                print(f"{cca1}-{cca2}-{tag}-red-{mult}bdp")
                
                applied_bdp = int(bdp * mult)
                packets = int(applied_bdp / mtu_setting) #mtu is 8900 bytes
                
                print(f"Configuring buffer on node {r1_name}")
                r1 = fabric_slice.get_node(name=r1_name)
        
                r1_iface_r2 = r1.get_interface(network_name=net_r1_r2_name)    
                r1_r2_os_iface = r1_iface_r2.get_os_interface()

                print(f"iface: {r1_r2_os_iface}, speed: {speed}, packets: {packets}")

                r1.execute(f'sudo tc qdisc del dev {r1_r2_os_iface} root', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} root handle 1: tbf rate {speed} burst 1570000 limit {applied_bdp}', quiet=False)
                r1.execute(f'sudo tc qdisc add dev {r1_r2_os_iface} parent 1:1 handle 10: red limit {applied_bdp} avpkt 1000 min 30000 max 30001 bandwidth {speed} ecn', quiet=False)
                    
                
                #run 5 reps
                for rep in range(1,2):
                    print(f"Starting run {rep}")
                    #start servers
                    print(f"Starting servers on node {h3_name}")
                    node = fabric_slice.get_node(name=h3_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca1} {cca2} red {mult}bdp {rep} {processes}", quiet=True)

                    print(f"Starting servers on node {h4_name}")
                    node = fabric_slice.get_node(name=h4_name)
                    stdout, stderr = node.execute(f"bash /home/ubuntu/start_iperf_server.sh {tag} {cca2} {cca1} red {mult}bdp {rep} {processes}", quiet=True)
                   
                    print(f"Starting queue script on router {r1_name}")
                    print(f'python3 /home/ubuntu/buffer_status.py {tag} {cca2} {cca1} red {mult}bdp {rep} {r1_r2_os_iface} {runtime+10} buffer_status.csv &')
                    stdout, stderr = r1.execute(f'python3 /home/ubuntu/buffer_status_red.py {tag} {cca1} {cca2} red {mult}bdp {rep} {r1_r2_os_iface} {runtime+10} > test.out 2>&1 &', quiet=True)
                    print(f"Executed queue script on router {r1_name}")
                    #print(stdout)
                    #print(stderr)

                    print(f"Starting active rtt probing in {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    stdout, stderr = node.execute(f'python3 /home/ubuntu/active_rtt_probing.py 192.168.2.2 192.168.4.2 {runtime} 100 {tag} {cca1} {cca2} red {mult}bdp {rep} > probe.out 2>&1 &', quiet=True)

                    print(f"Starting active rtt probing in {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    stdout, stderr = node.execute(f'python3 /home/ubuntu/active_rtt_probing.py 192.168.3.2 192.168.5.2 {runtime} 100 {tag} {cca2} {cca1} red {mult}bdp {rep} > probe.out 2>&1 &', quiet=True)
                    
                    cmd_host_1 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca1} {cca2} red {mult}bdp {rep} {processes} 192.168.4.2 {streams}"    
                    
                    cmd_host_2 = f"bash /home/ubuntu/start_iperf_client.sh {tag} {cca2} {cca1} red {mult}bdp {rep} {processes} 192.168.5.2 {streams}"

                    #start clients via execute threads
                    execute_threads = {}
                    print(f"Starting clients on node {h1_name}")
                    node = fabric_slice.get_node(name=h1_name)
                    execute_threads[node] = node.execute_thread(cmd_host_1)
                    print(f"Starting clients on node {h2_name}")
                    node = fabric_slice.get_node(name=h2_name)
                    execute_threads[node] = node.execute_thread(cmd_host_2)

                    #Wait for results from threads
                    for node,thread in execute_threads.items():
                        #print(f"Waiting for result from node {node.get_name()}")
                        stdout,stderr = thread.result()

                    #iperf runs for 200seconds so we wait for 210
                    print("Waiting for iperf3 to finish for 210seconds")
                    sleep(runtime)

                    print(f"Cleaning and killing iperf3 and tcp_dump")                
                    #kill all iperf processes
                    kill_iperf_command = "pkill -f iperf3"

                    #kill all active probing processes
                    kill_active_probing = "pkill -f active_rtt_probing.py"
                    
                    #kill all tcp_dump processes
                    kill_tcpdump_command = "sudo pkill -f tcpdump"
                    
                    #kill all tcp_dump processes
                    kill_buffer_status_command = "sudo pkill -f buffer_status_red.py"

                    for node_name in [h1_name, h2_name, h3_name, h4_name, r1_name]:
                        node = fabric_slice.get_node(name=node_name)
                        print(f"Killing iperf3 and tcp_dump on node {node.get_name()}")
                        stdout,stderr = node.execute(kill_iperf_command)                        
                        stdout, stderr = node.execute(kill_tcpdump_command)  # Stop tcpdump gracefully by sending SIGINT sign
                        stdout, stderr = node.execute(kill_buffer_status_command)
                    
                    
                    cmd_process_pcap_h1 = f"python3 /home/ubuntu/process_pcap.py /home/ubuntu/output/{tag}_{cca1}_{cca2}_red_{mult}bdp_{rep}"
                    cmd_process_pcap_h2 = f"python3 /home/ubuntu/process_pcap.py /home/ubuntu/output/{tag}_{cca2}_{cca1}_red_{mult}bdp_{rep}"
                   

                    node = fabric_slice.get_node(name=h1_name)
                    print(f"Starting pcap_process on node {node.get_name()}")
                    stdout, stderr = run_with_timeout(node, cmd_process_pcap_h1, 700)
                    if stdout is None and stderr is None:
                        print(f"Timeout or error occurred on node {node.get_name()} for command: {cmd_process_pcap_h1}")
                    else:
                        print(f"Completed pcap_process on node {node.get_name()}")

                    node = fabric_slice.get_node(name=h2_name)
                    print(f"Starting pcap_process on node {node.get_name()}")
                    stdout, stderr = run_with_timeout(node, cmd_process_pcap_h2, 700)
                    if stdout is None and stderr is None:
                        print(f"Timeout or error occurred on node {node.get_name()} for command: {cmd_process_pcap_h2}")
                    else:
                        print(f"Completed pcap_process on node {node.get_name()}")
                   
    print("All done with red!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [ ]:
output.show()

## Compress and download output logs

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for node_name in [h1_name, h2_name, h3_name, h4_name, r1_name]:
        tar_command = f"tar -czvf {node_name}_output.tar.gz output"
        node = fabric_slice.get_node(name=node_name)
        print(f"Compressing output on node {node.get_name()}")
        stdout,stderr = node.execute(tar_command, quiet=True)
        print(f"Downloading compressed output from node {node.get_name()}")
        node.download_file(f"{node_name}_output.tar.gz", f"{node_name}_output.tar.gz")


    #for node_name in [h1_name]:
    #    node = fabric_slice.get_node(name=node_name)
    #    print(f"Downloading compressed output from node {node.get_name()}")
    #    node.download_file(f"sender1_combined_output.tar.gz", f"sender1_combined_output.tar.gz")


    #for node_name in [h2_name]:
    #    node = fabric_slice.get_node(name=node_name)
    #    print(f"Downloading compressed output from node {node.get_name()}")
    #    node.download_file(f"combined_output.tar.gz", f"combined_output.tar.gz")
    
    print("Done!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Helpers

In [ ]:
try:
    kill_command = "pkill -f iperf3"
    kill_command_tcpdump = "sudo pkill -f tcpdump"
    kill_command_buffer_status = "pkill -f buffer_status.py"
    kill_active_probing = "pkill -f active_rtt_probing.py"
    
    fabric_slice = fablib.get_slice(name=slice_name)
    for node_name in [h1_name, h2_name, h3_name, h4_name]:
        node = fabric_slice.get_node(name=node_name)
        print(f"Killing iperf3 on node {node.get_name()}")
        stdout,stderr = node.execute(kill_command)
        stdout,stderr = node.execute(kill_command_tcpdump)
        stdout,stderr = node.execute(kill_command_buffer_status)
        stdout,stderr = node.execute(kill_active_probing)
    print("Done!")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

In [ ]:
from time import sleep

try:
    #start servers
    node = fabric_slice.get_node(name=h4_name)
    node.execute(f"ps -ax | grep iperf3")

    node = fabric_slice.get_node(name=h3_name)
    node.execute(f"ps -ax | grep iperf3")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

## Connect to Slice Nodes

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    for node in fabric_slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
#        print(f"   Interfaces        :  ")
#        for interface in node.get_interfaces():
#            print(f"       Name                : {interface.get_name()}")
#            print(f"           Bandwidth           : {interface.get_bandwidth()}")
#            print(f"           VLAN                : {interface.get_vlan()}")  
#            print(f"           MAC                 : {interface.get_mac()}") 
#            print(f"           OS iface name       : {interface.get_os_interface()}")
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()


## Delete Slice

In [ ]:
try:
    fabric_slice = fablib.get_slice(name=slice_name)
    fabric_slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()